[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/5_train_preference_comparisons.ipynb)
# Learning a Reward Function using Preference Comparisons

The preference comparisons algorithm learns a reward function by comparing trajectory segments to each other.

To set up the preference comparisons algorithm, we first need to set up a lot of its internals beforehand:

In [1]:
from imitation.algorithms import preference_comparisons
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
from imitation.policies.base import FeedForward32Policy, NormalizeFeaturesExtractor
from stable_baselines3 import PPO
import numpy as np

rng = np.random.default_rng(0)

venv = make_vec_env("Pendulum-v1", rng=rng)

reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)

fragmenter = preference_comparisons.RandomFragmenter(
    warning_threshold=0,
    rng=rng,
)
gatherer = preference_comparisons.SyntheticGatherer(rng=rng)
preference_model = preference_comparisons.PreferenceModel(reward_net)
reward_trainer = preference_comparisons.BasicRewardTrainer(
    preference_model=preference_model,
    loss=preference_comparisons.CrossEntropyRewardLoss(),
    epochs=3,
    rng=rng,
)


# Several hyperparameters (reward_epochs, ppo_clip_range, ppo_ent_coef,
# ppo_gae_lambda, ppo_n_epochs, discount_factor, use_sde, sde_sample_freq,
# ppo_lr, exploration_frac, num_iterations, initial_comparison_frac,
# initial_epoch_multiplier, query_schedule) used in this example have been
# approximately fine-tuned to reach a reasonable level of performance.
agent = PPO(
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=venv,
    seed=0,
    n_steps=2048 // venv.num_envs,
    batch_size=64,
    ent_coef=0.01,
    learning_rate=2e-3,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    n_epochs=10,
)

trajectory_generator = preference_comparisons.AgentTrainer(
    algorithm=agent,
    reward_fn=reward_net,
    venv=venv,
    exploration_frac=0.05,
    rng=rng,
)

pref_comparisons = preference_comparisons.PreferenceComparisons(
    trajectory_generator,
    reward_net,
    num_iterations=5, # Set to 60 for better performance
    fragmenter=fragmenter,
    preference_gatherer=gatherer,
    reward_trainer=reward_trainer,
    fragment_length=100,
    transition_oversampling=1,
    initial_comparison_frac=0.1,
    allow_variable_horizon=False,
    initial_epoch_multiplier=4,
    query_schedule="hyperbolic",
)

Then we can start training the reward model. Note that we need to specify the total timesteps that the agent should be trained and how many fragment comparisons should be made.

In [2]:
pref_comparisons.train(
    total_timesteps=5_000, 
    total_comparisons=200,
)

Query schedule: [20, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Requested 3800 transitions but only 0 in buffer. Sampling 3800 additional transitions.
Sampling 200 exploratory transitions.
Creating fragment pairs
Gathering preferences
Dataset now contains 20 comparisons


Training reward model:   0%|          | 0/12 [00:00<?, ?it/s]

Training agent for 83 timesteps
----------------------------------------------------
| raw/                                 |           |
|    agent/rollout/ep_len_mean         | 200       |
|    agent/rollout/ep_rew_mean         | -1.44e+03 |
|    agent/rollout/ep_rew_wrapped_mean | 33.1      |
|    agent/time/fps                    | 17650     |
|    agent/time/iterations             | 1         |
|    agent/time/time_elapsed           | 0         |
|    agent/time/total_timesteps        | 2048      |
----------------------------------------------------
-------------------------------------------------------
| mean/                                   |           |
|    agent/rollout/ep_len_mean            | 200       |
|    agent/rollout/ep_rew_mean            | -1.44e+03 |
|    agent/rollout/ep_rew_wrapped_mean    | 33.1      |
|    agent/time/fps                       | 1.76e+04  |
|    agent/time/iterations                | 1         |
|    agent/time/time_elapsed              | 0 

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 83 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_len_mean         | 200          |
|    agent/rollout/ep_rew_mean         | -1.26e+03    |
|    agent/rollout/ep_rew_wrapped_mean | 30.1         |
|    agent/time/fps                    | 13715        |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 0            |
|    agent/time/total_timesteps        | 4096         |
|    agent/train/approx_kl             | 0.0013779316 |
|    agent/train/clip_fraction         | 0.055        |
|    agent/train/clip_range            | 0.1          |
|    agent/train/entropy_loss          | -1.43        |
|    agent/train/explained_variance    | -0.82        |
|    agent/train/learning_rate         | 0.002        |
|    agent/train/loss                  | 0.0412       |
|    agent/train/n_updates             | 10           |
|    agent/train

...

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 103 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_len_mean         | 200          |
|    agent/rollout/ep_rew_mean         | -217         |
|    agent/rollout/ep_rew_wrapped_mean | 13.1         |
|    agent/time/fps                    | 18079        |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 0            |
|    agent/time/total_timesteps        | 122880       |
|    agent/train/approx_kl             | 0.0037307334 |
|    agent/train/clip_fraction         | 0.154        |
|    agent/train/clip_range            | 0.1          |
|    agent/train/entropy_loss          | -1.39        |
|    agent/train/explained_variance    | 0.808        |
|    agent/train/learning_rate         | 0.002        |
|    agent/train/loss                  | 2.5          |
|    agent/train/n_updates             | 590          |
|    agent/trai

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 83 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_len_mean         | 200          |
|    agent/rollout/ep_rew_mean         | -204         |
|    agent/rollout/ep_rew_wrapped_mean | 14.5         |
|    agent/time/fps                    | 17098        |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 0            |
|    agent/time/total_timesteps        | 124928       |
|    agent/train/approx_kl             | 0.0072140205 |
|    agent/train/clip_fraction         | 0.203        |
|    agent/train/clip_range            | 0.1          |
|    agent/train/entropy_loss          | -1.36        |
|    agent/train/explained_variance    | 0.831        |
|    agent/train/learning_rate         | 0.002        |
|    agent/train/loss                  | 3.17         |
|    agent/train/n_updates             | 600          |
|    agent/train

{'reward_loss': 0.051602873152920184, 'reward_accuracy': 0.9776785714285714}

After we trained the reward network using the preference comparisons algorithm, we can wrap our environment with that learned reward.

In [3]:
from imitation.rewards.reward_wrapper import RewardVecEnvWrapper

learned_reward_venv = RewardVecEnvWrapper(venv, reward_net.predict_processed)

Next, we train an agent that sees only the shaped, learned reward.

In [4]:
learner = PPO(
    seed=0,
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=learned_reward_venv,
    batch_size=64,
    ent_coef=0.01,
    n_epochs=10,
    n_steps=2048 // learned_reward_venv.num_envs,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    learning_rate=2e-3,
)
learner.learn(1_000)  # Note: set to 100_000 to train a proficient expert

Then we can evaluate it using the original reward.

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

n_eval_episodes = 10
reward_mean, reward_std = evaluate_policy(learner.policy, venv, n_eval_episodes)
reward_stderr = reward_std/np.sqrt(n_eval_episodes)
print(f"Reward: {reward_mean:.0f} +/- {reward_stderr:.0f}")

Reward: -139 +/- 28
